In [ ]:
import os
import warnings
import json
from typing import Tuple, Callable, Union
import nibabel
import numpy as np
import numba

from petpal.kinetic_modeling.fit_tac_with_rtms import (
                                                       get_rtm_method,
                                                       get_rtm_output_size)
from petpal.utils.time_activity_curve import TimeActivityCurveFromFile
from petpal.utils.image_io import safe_load_4dpet_nifti
from petpal.kinetic_modeling import graphical_analysis
from petpal.kinetic_modeling.reference_tissue_models import weight_tac_decay, weight_tac_simple
from petpal.input_function.blood_input import read_plasma_glucose_concentration
from petpal.utils.image_io import safe_load_tac, safe_copy_meta, validate_two_images_same_dimensions

In [5]:
def apply_rtm2_to_all_voxels(tac_times_in_minutes: np.ndarray,
                             tgt_image: np.ndarray,
                             ref_tac_vals: np.ndarray,
                             mask_img: np.ndarray,
                             method: str = 'srtm2',
                             **analysis_kwargs) -> np.ndarray:
    """
    Generates parametric images for 4D-PET data using the SRTM2 reference tissue method.

    Args:
        tac_times_in_minutes (np.ndarray): A 1D array representing the reference TAC and PET frame
            times in minutes.
        tgt_image (np.ndarray): A 4D array representing the 3D PET image over time.
            The shape of this array should be (x, y, z, time).
        ref_tac_vals (np.ndarray): A 1D array representing the reference TAC values. This array
            should be of the same length as `tac_times_in_minutes`.
        mask_img (np.ndarray): A 3D array representing the brain mask for `tgt_image`, where brain
            regions are labelled 1 and non-brain regions are labelled 0. This is made necessary in
            order to save time during computation. 

    Returns:
        params_img (np.ndarray): A 4D array with RTM parameter fit results based on the supplied
            method.
    """
    bounds = None
    if "bounds" in analysis_kwargs:
        bounds = True
    analysis_func = get_rtm_method(method=method,bounds=bounds)
    img_dims = tgt_image.shape
    output_shape = get_rtm_output_size(method=method)
    params_img = np.zeros((img_dims[0], img_dims[1], img_dims[2], output_shape), float)

    for i in range(0, img_dims[0], 1):
        for j in range(0, img_dims[1], 1):
            for k in range(0, img_dims[2], 1):
                if mask_img[i,j,k]>0.5:
                    analysis_vals = analysis_func(tac_times_in_minutes=tac_times_in_minutes,
                                                  ref_tac_vals=ref_tac_vals,
                                                  tgt_tac_vals=tgt_image[i, j, k, :],
                                                  **analysis_kwargs)
                    params_img[i,j,k] = analysis_vals[0]

    return params_img

In [ ]:
def apply_rtm2_to_all_voxels_weight_simple(tac_times_in_minutes: np.ndarray,
                                           tgt_image: np.ndarray,
                                           ref_tac_vals: np.ndarray,
                                           mask_img: np.ndarray,
                                           method: str = 'srtm2',
                                           **analysis_kwargs) -> np.ndarray:
    """
    Generates parametric images for 4D-PET data using the SRTM2 reference tissue method.

    Args:
        tac_times_in_minutes (np.ndarray): A 1D array representing the reference TAC and PET frame
            times in minutes.
        tgt_image (np.ndarray): A 4D array representing the 3D PET image over time.
            The shape of this array should be (x, y, z, time).
        ref_tac_vals (np.ndarray): A 1D array representing the reference TAC values. This array
            should be of the same length as `tac_times_in_minutes`.
        mask_img (np.ndarray): A 3D array representing the brain mask for `tgt_image`, where brain
            regions are labelled 1 and non-brain regions are labelled 0. This is made necessary in
            order to save time during computation. 

    Returns:
        params_img (np.ndarray): A 4D array with RTM parameter fit results based on the supplied
            method.
    """
    bounds = None
    if "bounds" in analysis_kwargs:
        bounds = True
    analysis_func = get_rtm_method(method=method,bounds=bounds)
    img_dims = tgt_image.shape
    output_shape = get_rtm_output_size(method=method)
    params_img = np.zeros((img_dims[0], img_dims[1], img_dims[2], output_shape), float)

    for i in range(0, img_dims[0], 1):
        for j in range(0, img_dims[1], 1):
            for k in range(0, img_dims[2], 1):
                if mask_img[i,j,k]>0.5:
                    analysis_vals = analysis_func(tac_times_in_minutes=tac_times_in_minutes,
                                                  ref_tac_vals=ref_tac_vals,
                                                  tgt_tac_vals=tgt_image[i, j, k, :],
                                                  **analysis_kwargs)
                    params_img[i,j,k] = analysis_vals[0]

    return params_img